In [1]:
import websocket
import json
import time
from tqdm import tqdm
from datetime import datetime, timedelta

message_count = 0
response_data = None  # Global variable to store response
fina_data = {}  # Store all fetched data globally
authToken = ''
workspaceID = ''
correlationId = ''
cookies = ''




def driver_get_tokens():
    from seleniumwire import webdriver
    global authToken,workspaceID,correlationId,cookies
    driver = webdriver.Chrome()
    today_date = datetime.today().strftime('%Y%m%d')
    driver.get(f"https://bexar.tx.publicsearch.us/results?department=RP&limit=50&offset=0&recordedDateRange=20200101%2C{today_date}&searchOcrText=true&searchType=quickSearch&searchValue=%22Foreclosure%22&viewType=card")
    time.sleep(2)
    for request in driver.requests:
        if "Cookie" in request.headers:
            if "authToken" in request.headers.get('cookie', None):
                cookies = request.headers.get('cookie', None)
        if request.response and 'Upgrade' in request.response.headers and request.response.headers['Upgrade'] == 'websocket':
            
            for message in request.ws_messages:
                if 'authToken' in message.content and "limit" in message.content and "workspaceID" in message.content:  # Capture messages sent by the browser
                    data = json.loads(message.content)
                    authToken = data["authToken"]
                    workspaceID = data["payload"]["workspaceID"]
                    correlationId = data["correlationId"]
                    print(authToken,workspaceID,correlationId)
                    break
    driver.close()
def on_open(ws):
    global start_date, end_date
    offset = ws.custom_offset  # Access custom attribute
    message = {
        'type': '@kofile/FETCH_DOCUMENTS/v4',
        'payload': {
            'query': {
                'limit': '250',
                'offset': str(offset),
                'department': 'RP',
                'recordedDateRange': f'{start_date.strftime("%Y%m%d")},{end_date.strftime("%Y%m%d")}',
                'searchOcrText': True,
                'searchType': 'quickSearch',
                'searchValue': 'Foreclosure'
            },
            'workspaceID': 'qgi33r8r3rq7m4f4zz8vc8'
        },
        'authToken': authToken,
        'correlationId': correlationId,
        'sync': True
    }
    ws.send(json.dumps(message))

def on_message(ws, message):
    global response_data
    response_data = message  # Store message in global variable
    ws.close()

def on_error(ws, error):
    print("### Error ###")
    print(error)
    print("Reconnecting...")
    time.sleep(5)  # Wait for 5 seconds before retrying
    try:
        ws.run_forever()
    except Exception as e:
        print("Reconnection failed:", e)

def on_close(ws, close_status_code, close_msg):
    pass

def create_websocket(offset):
    global cookies
    ws = websocket.WebSocketApp(
        "wss://bexar.tx.publicsearch.us/ws",
        on_open=lambda ws: on_open(ws),
        on_message=on_message,
        on_error=on_error,
        on_close=on_close,
        cookie=cookies
    )
    ws.custom_offset = offset  # Attach custom attribute for offset
    return ws

if __name__ == "__main__":
    #Geting tockens automatically
    driver_get_tokens()
    
    websocket.enableTrace(False)  # Set to True for debugging
    start_date = datetime.strptime("20241201", "%Y%m%d")
    end_today = datetime.now()

    while start_date < end_today:
        # Calculate the end of the current month
        end_date = start_date + timedelta(days=20)
        if end_date > end_today:
            end_date = end_today

        # Process records for the current month
        offset = 0
        ws = create_websocket(offset)
        ws.run_forever()

        if response_data:
            try:
                json_response = json.loads(response_data)
                data_docs = json_response["payload"]["data"]["byHash"]
                total_records = json_response["payload"]["meta"]["numRecords"]
                print("Total_rec ",total_records)
                if total_records > 10000:
                    end_date = end_date - timedelta(days=13)
                    ws = create_websocket(offset)
                    ws.run_forever()
                    json_response = json.loads(response_data)
                    data_docs = json_response["payload"]["data"]["byHash"]
                    total_records = json_response["payload"]["meta"]["numRecords"]
                    print("Total_rec2 ",total_records)
                fina_data.update(data_docs)
                print(f"Records collected: {len(fina_data)}")
            except json.JSONDecodeError:
                print("Error: Unable to decode JSON response.")
        else:
            print("No response received.")

        # Fetch remaining records for the month
        for offset in tqdm(range(250, total_records, 250), desc="Fetching records"):
            ws = create_websocket(offset)
            ws.run_forever()

            if response_data:
                try:
                    json_response = json.loads(response_data)
                    data_docs = json_response["payload"]["data"]["byHash"]
                    fina_data.update(data_docs)
                    print(f"Records collected: {len(fina_data)}")
                except json.JSONDecodeError:
                    print("Error: Unable to decode JSON response.")
            else:
                print("No response received.")

        # Move to the next month
        start_date = end_date + timedelta(days=1)
    final_time = datetime.now()
    print(final_time - end_today)
    print("### Data Collection Complete ###")
    print(f"Total records collected: {len(fina_data)}")


c25c9d86-0759-48ee-9c93-42994c7cc836 chfdrcvtwmkysb0a6nv8bl affd6b5a-347b-4a8a-871d-a457c91e2c31
Total_rec  3138
Records collected: 250


Fetching records:   8%|█████▍                                                           | 1/12 [00:06<01:15,  6.82s/it]

Records collected: 500


Fetching records:  17%|██████████▊                                                      | 2/12 [00:12<01:02,  6.24s/it]

Records collected: 750


...

Records collected: 7059


Fetching records:  86%|████████████████████████████████████████████████████████▌         | 6/7 [00:44<00:07,  7.75s/it]

Records collected: 7308


Fetching records: 100%|██████████████████████████████████████████████████████████████████| 7/7 [00:53<00:00,  7.68s/it]

Records collected: 7470
0:04:20.377503
### Data Collection Complete ###
Total records collected: 7470


In [ ]:
print("hello")

In [5]:
df = df.fillna('')


In [7]:
df.to_csv("BexarData1-30-2025.csv",index=False)

In [6]:
start_date

datetime.datetime(2021, 4, 7, 0, 0)

In [7]:
len(fina_data)

117045